In [ ]:
from google.colab import drive
drive.mount('/content/drive')
'''
!cp drive/MyDrive/optimizers.py .
!cp drive/MyDrive/yellowfin.py .

from optimizers import DemonRanger
from yellowfin import YFOptimizer
'''
from sklearn.model_selection import KFold
import sklearn
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torchvision
from torch.nn import functional as F

import numpy as np
import time 

import itertools
from tqdm.notebook import tqdm

from torch.optim.lr_scheduler import StepLR

import matplotlib.pyplot as plt

import torchvision.datasets as datasets

!pip install CosineAnnealingWithRestartsLR
from torch.optim import Adam
from torch.optim.lr_scheduler import LambdaLR
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim.lr_scheduler import LinearLR
from torch.optim.lr_scheduler import ExponentialLR

! [ ! -z "$COLAB_GPU" ] && pip install torch scikit-learn==0.20.* skorch

torch.manual_seed(42)

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement CosineAnnealingWithRestartsLR (from versions: none)
ERROR: No matching distribution found for CosineAnnealingWithRestartsLR
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.4 MB 7.5 MB/s 
     |████████████████████████████████| 155 kB 60.0 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but you have scikit-learn 0.20.4 which is incompatible.
imbalanced-learn 0.8.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
    if hasattr(layer, 'reset_parameters'):
      # print(f'Reset trainable parameters of layer = {layer}')
      layer.reset_parameters()

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.linear1 = nn.Linear(28*28, 256) 
        self.linear2 = nn.Linear(256, 100) 
        self.final = nn.Linear(100, 10)
        self.relu = nn.ReLU()

    def forward(self, img):
        x = img.view(-1, 28*28)
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        x = self.final(x)
        return x

In [ ]:
def train(model, data_loader, criterion, optimizer, mini_batch_size, view_every = 500):
    running_loss = 0.0

    for i, data in enumerate(data_loader, 0):
        
        # get the input data
        inputs = data[0].to(device)
        labels = data[1].to(device)
        
        # zero the gradients
        optimizer.zero_grad()

        # forward + backward + optimizte
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % view_every == view_every-1: 
            print('Batch {:d} / {:d}: loss = {:.4f}'.format(i+1,len(data_loader),running_loss/view_every))
            running_loss = 0.0

In [ ]:
def test(model, data_loader):
    correct = 0.0
    total = 0.0
        
    with torch.no_grad():
      for data in data_loader:
        images = data[0].to(device)
        labels = data[1].to(device)
        
        # compute outputs by running images through the network
        outputs = model(images)
        
        # choosing class with highest probability          
        pdf = F.softmax(outputs, dim=1) 
        predicted = pdf.argmax(dim=1)     

        correct += (predicted ==  labels).sum()
        total += len(predicted)
    return correct/total 

In [ ]:
def hyperparameters(model, params):
  start_time = time.time()
  keys, values = zip(*params.items())
  permutations_dict = [dict(zip(keys, v)) for v in itertools.product(*values)]

  opt_acc = 0.0
  opt_config = {}

  for config in tqdm(permutations_dict):
    print('-----------------------')
    print("Current Parameters: ", config)
    acc = cross_validation_accuracy(model, config)

    if acc > opt_acc:
      opt_acc = acc
      opt_config = config
  
  end_time = time.time()
  print(f" Best Accuracy: {opt_acc}, with parameters {opt_config}")
  print(f" Runtime: {((end_time - start_time)/60):.4} minutes")
  return opt_config, opt_acc

In [ ]:
def cross_validation_accuracy(model, params):
  # Define the K-fold Cross Validator
  kfold = KFold(n_splits = params["folds"], shuffle = True)

  # dict to save results
  accuracies = {}

  for fold, (train_idx, val_idx) in enumerate(kfold.split(mnist_train)):
    print(f"-------------Fold {fold+1}----------------")

    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
    val_subsampler = torch.utils.data.SubsetRandomSampler(val_idx)

    # Define the data loaders for training and testing data in this fold
    train_loader = torch.utils.data.DataLoader(mnist_train, 
                                               batch_size = params["batch_size"], 
                                               sampler = train_subsampler, 
                                               pin_memory = torch.cuda.is_available())
    val_loader = torch.utils.data.DataLoader(mnist_train, 
                                             batch_size = params["batch_size"], 
                                             sampler = val_subsampler, 
                                             pin_memory = torch.cuda.is_available())

    # Initialize the neural network
    model.apply(reset_weights)

    # Initialize model's parameters
    criterion = params["criterion"]()
    optimizer = params["optimizer"](model.parameters(), lr = params["lr"],momentum = params["momentum"], nesterov = params["nesterov"])
    #scheduler = params["scheduler"](optimizer, **params["scheduler_parameters"])

    val_acc = 0.0
    for epoch in range(params["epochs"]):
      train(model, train_loader, criterion, optimizer, params["batch_size"])
      curr_acc = (test(model, val_loader)) * 100.0
      print(f"epoch {epoch + 1}/{params['epochs']}: Accuracy: {curr_acc:.4}")
      val_acc = curr_acc

      '''try:
        scheduler.step()
      except:
        scheduler.step(curr_acc)'''

    accuracies[fold] = curr_acc

  # Print fold results
  print('--------------------------------')
  print(f'K-FOLD CROSS VALIDATION RESULTS FOR {params["folds"]} FOLDS')
  print('--------------------------------')

  sum = 0.0
  for fold, value in accuracies.items():
    print(f"Fold {fold}: {value} %")
    sum += value
  
  avg_acc = sum/len(accuracies.items())
  print(f"Average: {avg_acc:.4} %")
  print("-----------")
  
  return avg_acc

### Data & Loaders

In [ ]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

mnist_train = datasets.MNIST(root='./data', 
                             train=True, 
                             download=True, 
                             transform=transform)
mnist_test = torchvision.datasets.MNIST('./data/files/', 
                                        train=False, 
                                        download=True,  
                                        transform=transform)

train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=64, shuffle=True, pin_memory=torch.cuda.is_available())
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=64, shuffle=True, pin_memory = torch.cuda.is_available())

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

...

Extracting ./data/files/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/files/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/files/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/files/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/files/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/files/MNIST/raw



In [ ]:
mnist_train.data.shape, mnist_test.data.shape

(torch.Size([60000, 28, 28]), torch.Size([10000, 28, 28]))

### SGDM

In [ ]:


parameters = {
    "criterion": [torch.nn.CrossEntropyLoss],
    "lr": [0.002, 0.02, 0.2],
    "momentum":[0.9],
    "nesterov":[False],
    "optimizer": [torch.optim.SGD],
    "batch_size": [90, 120, 180],
    "folds": [5],
    "epochs": [25]}

model = Net()
model.to(device)

opt_config_lambda, opt_acc_lambda = hyperparameters(model, parameters)

  0%|          | 0/9 [00:00<?, ?it/s]

-----------------------
Current Parameters:  {'criterion': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'lr': 0.002, 'momentum': 0.9, 'nesterov': False, 'optimizer': <class 'torch.optim.sgd.SGD'>, 'batch_size': 90, 'folds': 5, 'epochs': 25}
-------------Fold 1----------------
Batch 500 / 534: loss = 1.7428
epoch 1/25: Accuracy: 82.81
Batch 500 / 534: loss = 0.5168
epoch 2/25: Accuracy: 88.26
Batch 500 / 534: loss = 0.3780
epoch 3/25: Accuracy: 90.01
Batch 500 / 534: loss = 0.3324
epoch 4/25: Accuracy: 91.07
Batch 500 / 534: loss = 0.3004
epoch 5/25: Accuracy: 91.66
Batch 500 / 534: loss = 0.2749
epoch 6/25: Accuracy: 92.36
Batch 500 / 534: loss = 0.2528
epoch 7/25: Accuracy: 92.78
Batch 500 / 534: loss = 0.2349
epoch 8/25: Accuracy: 93.24
Batch 500 / 534: loss = 0.2183
epoch 9/25: Accuracy: 93.58
Batch 500 / 534: loss = 0.2034
epoch 10/25: Accuracy: 93.98
Batch 500 / 534: loss = 0.1907
epoch 11/25: Accuracy: 94.25
Batch 500 / 534: loss = 0.1786
epoch 12/25: Accuracy: 94.45
Batch 5

ValueError: ignored

### SGD-Nesterov

In [ ]:
parameters = {
    "criterion": [torch.nn.CrossEntropyLoss],
    "lr": [0.002, 0.05, 0.2],
    "momentum":[0.9],
    "nesterov":[True],
    "optimizer": [torch.optim.SGD],
    "batch_size": [90, 120, 180],
    "folds": [5],
    "epochs": [25]} 

model = Net()
model.to(device)

opt_config_cos, opt_acc_cos = hyperparameters(model, parameters)


  0%|          | 0/9 [00:00<?, ?it/s]

-----------------------
Current Parameters:  {'criterion': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'lr': 0.002, 'momentum': 0.9, 'nesterov': True, 'optimizer': <class 'torch.optim.sgd.SGD'>, 'batch_size': 90, 'folds': 5, 'epochs': 25}
-------------Fold 1----------------
Batch 500 / 534: loss = 1.6714
epoch 1/25: Accuracy: 82.42
Batch 500 / 534: loss = 0.5079
epoch 2/25: Accuracy: 87.92
Batch 500 / 534: loss = 0.3750
epoch 3/25: Accuracy: 89.74
Batch 500 / 534: loss = 0.3292
epoch 4/25: Accuracy: 90.5
Batch 500 / 534: loss = 0.3016
epoch 5/25: Accuracy: 91.2
Batch 500 / 534: loss = 0.2767
epoch 6/25: Accuracy: 91.82
Batch 500 / 534: loss = 0.2571
epoch 7/25: Accuracy: 92.18
Batch 500 / 534: loss = 0.2403
epoch 8/25: Accuracy: 92.73
Batch 500 / 534: loss = 0.2242
epoch 9/25: Accuracy: 93.18
Batch 500 / 534: loss = 0.2091
epoch 10/25: Accuracy: 93.63
Batch 500 / 534: loss = 0.1970
epoch 11/25: Accuracy: 94.02
Batch 500 / 534: loss = 0.1850
epoch 12/25: Accuracy: 94.07
Batch 500 

ValueError: ignored

### SGD

In [ ]:
parameters = {
    "criterion": [torch.nn.CrossEntropyLoss],
    "lr": [0.002, 0.05, 0.2],
    "optimizer": [torch.optim.SGD],
    "batch_size": [90, 120, 180],
    "folds": [5],
    "epochs": [25],
   }

model = Net()
model.to(device)

opt_config_linear, opt_acc_linear = hyperparameters(model, parameters)

  0%|          | 0/9 [00:00<?, ?it/s]

-----------------------
Current Parameters:  {'criterion': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'lr': 0.002, 'optimizer': <class 'torch.optim.sgd.SGD'>, 'batch_size': 90, 'folds': 5, 'epochs': 25}
-------------Fold 1----------------
Batch 500 / 534: loss = 2.2850
epoch 1/25: Accuracy: 31.25
Batch 500 / 534: loss = 2.2295
epoch 2/25: Accuracy: 48.56
Batch 500 / 534: loss = 2.1329
epoch 3/25: Accuracy: 56.22
Batch 500 / 534: loss = 1.9581
epoch 4/25: Accuracy: 66.61
Batch 500 / 534: loss = 1.6848
epoch 5/25: Accuracy: 69.87
Batch 500 / 534: loss = 1.3693
epoch 6/25: Accuracy: 72.72
Batch 500 / 534: loss = 1.1065
epoch 7/25: Accuracy: 75.64
Batch 500 / 534: loss = 0.9215
epoch 8/25: Accuracy: 78.62
Batch 500 / 534: loss = 0.7997
epoch 9/25: Accuracy: 80.48
Batch 500 / 534: loss = 0.7127
epoch 10/25: Accuracy: 81.85
Batch 500 / 534: loss = 0.6465
epoch 11/25: Accuracy: 83.12
Batch 500 / 534: loss = 0.5987
epoch 12/25: Accuracy: 84.07
Batch 500 / 534: loss = 0.5618
epoch 13/25:

ValueError: ignored